In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from astropy.table import Table
import os
import numpy as np
import cartopy.crs as ccrs
import json
import os
import seaborn as sns
from astropy.table import Table

In [2]:
os.chdir('/home/walml/repos/zoobot')

In [3]:
# 'Fixed' classifications from *_fixes.ipynb
dr2_c = pd.read_csv('dr2_aggregated_votes_good_subjects.csv')  # now also includes subject info, like DR5
# will get from NSA again anyway
del dr2_c['ra']
del dr2_c['dec']
del dr2_c['redshift']

dr5_c = pd.read_csv('/home/walml/repos/zoobot/current_final_dr5_result_without_metadata.csv')



/home/walml/anaconda3/envs/zoobot/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (93) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Merge DR1/2 subject catalog with iauname, ra, dec, from NSA. DR5 already has this.


In [4]:

nsa = Table.read('/media/walml/beta/galaxy_zoo/decals/catalogs/raw/nsa_v1_0_0.fits')
nsa = nsa[['RA', 'DEC', 'IAUNAME', 'PETROTHETA', 'PETROTH50', 'PETROTH90', 'ABSMAG', 'RNMGY', 'Z']]

nsa['ABSMAG_R'] = nsa['ABSMAG'][:, 4]
nsa['RNMGY_R'] = nsa['RNMGY'][:, 4]
del nsa['ABSMAG']
del nsa['RNMGY']

nsa['REDSHIFT'] = nsa['Z']
del nsa['Z']

nsa = nsa.to_pandas()
for col in nsa.columns.values:
    nsa[col.lower()] = nsa[col]
    del nsa[col]

nsa['iauname'] = nsa['iauname'].str.decode('utf8')

In [5]:
nsa.head()

,ra,dec,iauname,petrotheta,petroth50,petroth90,absmag_r,rnmgy_r,redshift
0,146.714215,-1.041280,J094651.40-010228.5,7.247893,3.464192,10.453795,-19.664631,1726.364258,0.021222
1,146.628585,-0.765162,J094630.85-004554.5,5.617822,2.326989,6.721991,-20.027859,226.369293,0.064656
2,146.631706,-0.988347,J094631.59-005917.7,4.769891,2.278736,5.177910,-18.398241,82.433693,0.052654
3,146.934129,-0.670405,J094744.18-004013.4,6.243227,2.655191,9.177605,-21.748932,266.264313,0.121270
4,146.963500,-0.544776,J094751.74-003242.0,8.891541,4.383704,10.253274,-19.075111,138.427261,0.055981


In [6]:
# metadata_cols = ['iauname', 'ra', 'dec', 'petrotheta', 'petroth50', 'petroth90', 'rnmgy_r', 'redshift']
dr2 = pd.merge(dr2_c, nsa, on='iauname', how='inner')
assert len(dr2) == len(dr2_c)  # all must match exactly once
assert not any(dr2['subject_id'].duplicated())
assert not any(dr2['iauname'].duplicated())

In [7]:
dr2.iloc[0][-10:]  # 

dr2_jpeg_filled           False
good_subject_image         True
ra                       158.66
dec                    -0.85268
petrotheta              5.98512
petroth50               2.88914
petroth90               6.54693
absmag_r               -21.1833
rnmgy_r                 514.105
redshift              0.0737827
Name: 0, dtype: object

In [8]:
dr2['active_learning_on'] = False
dr2['upload_group'] = 'pre_active'

In [9]:
dr2

,subject_id,smooth-or-featured_smooth,smooth-or-featured_featured-or-disk,smooth-or-featured_artifact,how-rounded_completely,how-rounded_in-between,how-rounded_cigar-shaped,disk-edge-on_yes,disk-edge-on_no,edge-on-bulge_rounded,...,ra,dec,petrotheta,petroth50,petroth90,absmag_r,rnmgy_r,redshift,active_learning_on,upload_group
0,55a7d7a27a726119cd00000b,19,15,11,16,2,1,2,13,2,...,158.659544,-0.852680,5.985117,2.889140,6.546928,-21.183338,514.105469,0.073783,False,pre_active
1,55a7d7a27a726119cd00000c,33,2,6,28,5,0,1,1,1,...,158.928432,-0.752741,3.493677,1.599772,4.897500,-20.991505,156.159576,0.115605,False,pre_active
2,55a7d7a27a726119cd00000d,36,1,10,23,13,0,0,1,0,...,158.757099,-0.831234,2.698255,1.326934,3.000635,-19.948416,57.348721,0.118851,False,pre_active
3,55a7d7a27a726119cd00000e,32,15,6,16,16,0,0,15,0,...,159.207820,-0.631470,6.951218,3.622341,7.018666,-19.635822,173.241699,0.063560,False,pre_active
4,55a7d7a27a726119cd00000f,40,6,7,10,29,1,0,6,0,...,159.196722,-0.682358,4.589167,2.078999,5.593759,-20.093359,102.514648,0.095834,False,pre_active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92955,56f3deb65925d90042037015,6,12,22,4,1,1,0,12,0,...,217.692900,-1.673523,10.527441,5.647549,11.515634,-18.743391,105.568184,0.055447,False,pre_active
92956,56f3deb65925d90042037017,21,15,4,17,4,0,0,15,0,...,217.054418,-1.929178,6.721991,3.366963,9.033441,-18.780859,109.207932,0.054090,False,pre_active
92957,56f3deb65925d90042037019,22,15,3,2,16,4,11,4,11,...,216.944227,-1.432235,5.407866,2.608586,8.073513,-21.928631,229.591217,0.138307,False,pre_active
92958,56f3deb65925d9004203701b,30,2,8,26,3,1,0,2,0,...,216.924991,-1.755687,15.874659,5.831343,21.499472,-21.752384,1717.734741,0.053837,False,pre_active


In [10]:
# dr2.to_csv('current_final_dr2_result.csv', index=False)

In [11]:
# Final number: 92,960 galaxies

## Repeat the same process with DR5. Classifications already include some metadata, so no need for subject catalog.

In [12]:

metadata_cols = ['iauname', 'petrotheta', 'petroth50', 'petroth90', 'rnmgy_r']  # ra, dec, redshift already included
dr5 = pd.merge(dr5_c, nsa[metadata_cols], on='iauname', how='inner')
assert len(dr5) == len(dr5_c)

In [13]:
dr5['upload_group'].value_counts()

active_random               163821
pre_active                   50946
active_priority               6999
targeted                      1387
calibration_nair_dr2_dr5      1262
Name: upload_group, dtype: int64

In [14]:
dr5['active_learning_on'] = dr5['upload_group'].isin(['active_random', 'active_priority')

SyntaxError: invalid syntax (<ipython-input-14-29a11c9127f6>, line 1)

In [ ]:
dr5

In [ ]:
# dr5.to_csv('current_final_dr5_result.csv', index=False)

In [ ]:
from matplotlib.ticker import FuncFormatter

In [ ]:
def custom_fmt(x, pos):
    if x == 5000:
        return ' 5,000'  # extra space for lining up y axis
    if x == 50000:
        return '50,000'
    else:
        return '0'

In [ ]:
comma_fmt = FuncFormatter(custom_fmt)

In [ ]:
color_cycle = ['#1f77b4', '#ff7f0e', '#2ca02c']

In [ ]:
sns.set_context('notebook')
sns.set(font_scale=1.5)
sns.set_style('white')
_, bins = np.histogram(np.linspace(0, 100), bins=60)
fig, axes = plt.subplots(nrows=3, figsize=(8, 6), sharex=True)
axes[0].hist(dr2_with_data_release.query('data_release == "DR1"')['smooth-or-featured_total-votes'], bins=bins, color=color_cycle[0])
axes[0].legend(['GZD-1'])
axes[1].hist(dr2_with_data_release.query('data_release == "DR2"')['smooth-or-featured_total-votes'], bins=bins, color=color_cycle[1])
axes[1].legend(['GZD-2'])
axes[2].hist(dr5['smooth-or-featured_total-votes'], bins=bins, color=color_cycle[2])
axes[2].legend(['GZD-5'])
for ax in axes:
    ax.set_xlim(0, 100)
    ax.set_ylabel('Galaxies')
axes[2].set_xlabel('Classifications per galaxy')
axes[0].yaxis.set_major_formatter(comma_fmt)
axes[1].yaxis.set_major_formatter(comma_fmt)
axes[2].yaxis.set_major_formatter(comma_fmt)
fig.tight_layout()
fig.savefig('latest_totals_per_dr.png')
fig.savefig('latest_totals_per_dr.pdf')

In [ ]:
(dr5['smooth-or-featured_total-votes'] > 45).mean()

In [ ]:
plt.hist(dr5['smooth-or-featured_total-votes'])

In [ ]:
dr2_classified_iaunames = dr2_s['iauname']

In [ ]:
dr5['in_dr2'] = dr5['iauname'].isin(dr2_classified_iaunames)

In [ ]:
dr5['in_dr2'].value_counts()